In [12]:
# 구글 이미지를 크롤링하는 코드 작성 (셀레니움 및 웹드라이버 매니저 이용)

# 검색할 단어 입력 (여기서 검색어를 수정하면 됩니다)
SEARCH_QUERY = "고양이"  # 원하는 검색어를 입력하세요

import time
import os
import requests
from datetime import datetime
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.action_chains import ActionChains
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import random

# 크롬 드라이버 설정 함수
def setup_chrome_driver():
    chrome_options = Options()
    # chrome_options.add_argument("--headless")  # 브라우저를 표시하지 않고 실행
    service = Service(ChromeDriverManager().install())
    driver = webdriver.Chrome(service=service, options=chrome_options)
    return driver

# 구글 이미지 검색어 입력 함수
def search_google_images(driver, search_query):
    try:
        driver.get("https://www.google.com/imghp")
        print("구글 이미지 검색 페이지에 접속했습니다.")
        
        search_box = driver.find_element(By.NAME, "q")
        search_box.send_keys(search_query)
        search_box.send_keys(Keys.RETURN)
        print(f"'{search_query}' 검색을 실행했습니다.")
        time.sleep(random.uniform(3, 5))  # 대기 시간을 랜덤하게 설정하여 크롤링 탐지 방지
    except Exception as e:
        print(f"검색어 입력 중 오류 발생: {e}")

# 이미지 URL 수집 함수
def collect_image_urls(driver, num_images):
    image_urls = set()
    wait = WebDriverWait(driver, 20)  # 대기 시간 늘림
    scroll_count = 0  # 스크롤 횟수 설정
    max_scrolls = 2

    while len(image_urls) < num_images and scroll_count < max_scrolls:
        try:
            wait.until(EC.presence_of_all_elements_located((By.CSS_SELECTOR, "img.rg_i")))
            thumbnails = driver.find_elements(By.CSS_SELECTOR, "img.rg_i")
            print(f"찾은 썸네일 개수: {len(thumbnails)}")  # 썸네일 개수 출력
            
            if len(thumbnails) == 0:
                print("썸네일을 찾을 수 없습니다. 페이지가 제대로 로드되지 않았을 수 있습니다.")

            for thumbnail in thumbnails[len(image_urls):]:
                try:
                    driver.execute_script("arguments[0].scrollIntoView(true);", thumbnail)
                    time.sleep(random.uniform(0.5, 1.0))  # 클릭 전에 짧은 대기

                    thumbnail.click()
                    # 이미지가 로드될 때까지 기다림
                    wait.until(EC.presence_of_element_located((By.CSS_SELECTOR, "img.n3VNCb")))
                    time.sleep(random.uniform(1, 2))  # 대기 시간을 랜덤하게 설정하여 크롤링 탐지 방지
                    
                    images = driver.find_elements(By.CSS_SELECTOR, "img.n3VNCb")
                    for image in images:
                        src = image.get_attribute("src")
                        if src and "http" in src:
                            image_urls.add(src)
                            print(f"이미지 URL 수집: {src}")
                        if len(image_urls) >= num_images:
                            break
                except Exception as e:
                    print(f"썸네일 클릭 중 오류 발생: {e}")

            # 페이지 스크롤 다운 (최대 2번만 스크롤)
            if scroll_count < max_scrolls:
                driver.find_element(By.TAG_NAME, "body").send_keys(Keys.END)
                time.sleep(random.uniform(2, 3))  # 대기 시간을 랜덤하게 설정하여 크롤링 탐지 방지
                scroll_count += 1

        except Exception as e:
            print(f"이미지 URL 수집 중 오류 발생: {e}")
    
    return image_urls

# 이미지 다운로드 함수
def download_images(image_urls):
    current_date = datetime.now().strftime("%Y%m%d")
    download_folder = f"사진다운로드_{current_date}"
    if not os.path.exists(download_folder):
        os.makedirs(download_folder)
    
    for idx, url in enumerate(image_urls):
        try:
            response = requests.get(url, stream=True, timeout=5)  # 서버 부하를 줄이기 위해 타임아웃 설정
            if response.status_code == 200:
                file_path = os.path.join(download_folder, f"image_{idx + 1}.jpg")
                with open(file_path, 'wb') as file:
                    for chunk in response.iter_content(1024):
                        file.write(chunk)
                print(f"이미지 저장 완료: {file_path}")
            else:
                print(f"이미지 다운로드 실패 (상태 코드: {response.status_code}): {url}")
        except Exception as e:
            print(f"이미지 다운로드 중 오류 발생: {e}")

# 크롬 드라이버 종료 함수
def close_driver(driver):
    driver.quit()

# 전체 구글 이미지 크롤링 함수
def google_image_crawler(search_query, num_images):
    driver = setup_chrome_driver()
    search_google_images(driver, search_query)
    image_urls = collect_image_urls(driver, num_images)
    close_driver(driver)
    
    # 수집된 이미지 URL 출력
    print(f"총 {len(image_urls)}개의 이미지를 수집하였습니다.")
    return image_urls

if __name__ == "__main__":
    num_images = 3  # 수집할 이미지 개수 (서버 부하를 줄이기 위해 적은 수로 설정)
    image_urls = google_image_crawler(SEARCH_QUERY, num_images)
    download_images(image_urls)


구글 이미지 검색 페이지에 접속했습니다.
'고양이' 검색을 실행했습니다.
이미지 URL 수집 중 오류 발생: Message: 

이미지 URL 수집 중 오류 발생: Message: 

이미지 URL 수집 중 오류 발생: Message: 

이미지 URL 수집 중 오류 발생: Message: 

이미지 URL 수집 중 오류 발생: Message: 

이미지 URL 수집 중 오류 발생: Message: 

이미지 URL 수집 중 오류 발생: Message: 



KeyboardInterrupt: 